In [ ]:
"""
import pandas as pd

# Load the CSV file into a pandas dataframe
df = pd.read_csv("/kaggle/input/aisbrest/ais_brest_locations.csv")

# Fetch all rows where the ship ID is 227362110
ship_data = df[df['id'] == 227362110]

# Print the resulting dataframe
print(ship_data)
"""

# Importing ship data

In [ ]:
"""
import pandas as pd
import numpy as np

# Load the CSV file into a pandas dataframe
df = pd.read_csv("/kaggle/input/aisbrest/ais_brest_locations.csv")

# Fetch all rows where the ship ID is 227362110
#ship_data = df[df['id'] == 227362110] #2008
#ship_data = df[df['id'] == 245334000] #10.000
ship_data = df[df['id'] ==227635210] #22347 --default
#ship_data = df[df['id'] ==227306100] #24320
#ship_data = df[df['id'] ==227574020] #54367


# Select only the desired columns and sort by timestamp
desired_columns = ['timestamp','longitude', 'latitude']
ship_data = ship_data[desired_columns].sort_values(by='timestamp')

print(ship_data)

#ship_data_matrix = ship_data.to_numpy()

# Set the display format of NumPy arrays
#np.set_printoptions(formatter={'float': lambda x: "{:.8f}".format(x)})

# Print the resulting matrix
#print(ship_data_matrix)
"""

In [ ]:
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV file into a pandas dataframe
df = pd.read_csv("/kaggle/input/aisbrest/ais_brest_locations.csv")

ship_data = df[df['id'] ==227635210] #22347 --default

ship_data = ship_data[ship_data['latitude'] <= 50]
ship_data = ship_data[ship_data['longitude'] >= -5]
# Select only the desired columns and sort by timestamp
ship_data = ship_data.sort_values(by='timestamp')


# Select only the desired columns and sort by timestamp
#desired_columns = ['timestamp','longitude', 'latitude']
#ship_data = ship_data[desired_columns].sort_values(by='timestamp')

print(ship_data)

fig, ax = plt.subplots(figsize=(15, 6))
# Create scatterplot between longitude and speed
plt.scatter(ship_data['longitude'], ship_data['speed'])
plt.xlabel('Speed')
plt.ylabel('Longitude')
plt.title('Scatterplot of Longitude and Speed')
plt.show()

fig, ax = plt.subplots(figsize=(15, 6))
# Create scatterplot between latitude and speed
plt.scatter(ship_data['latitude'], ship_data['speed'])
plt.xlabel('Speed')
plt.ylabel('Latitude')
plt.title('Scatterplot of Latitude and Speed')
plt.show()

# Select only the desired columns and the first 1000 rows
#ship_data_1000 = ship_data[['longitude', 'speed']][:1000]

# Create the scatterplot
#plt.scatter(ship_data_1000['longitude'], ship_data_1000['speed'])
#plt.xlabel('Speed')
#plt.ylabel('Longitude')
#plt.show()
"""

# Data evaluation

In [ ]:
"""
import numpy as np

ship_data = df[df['id'] ==227635210] #22347
#ship_data = df[df['id'] == 245334000] #10.000

ship_data = ship_data[ship_data['latitude'] <= 50]
ship_data = ship_data[ship_data['longitude'] >= -5]
# Select only the desired columns and sort by timestamp
desired_columns = ['timestamp','longitude', 'latitude']
ship_data = ship_data[desired_columns].sort_values(by='timestamp')

print(ship_data)
#22285 

# Convert longitude and latitude columns to numpy arrays
longitude = ship_data['longitude'].values
latitude = ship_data['latitude'].values

#print(longitude)
#print(latitude)

# Calculate Pearson's correlation coefficient
pearsons_coefficient = np.corrcoef(ship_data['longitude'], ship_data['latitude'])

print("\nThe Pearson's correlation coefficient of the longitude and latitude columns is:\n", pearsons_coefficient)
"""

In [ ]:
#from decimal import Decimal

#float_array = ship_data.astype(float)
#print(float_array)

# test 1

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm


# Define the models
models = {
    #lstm': lambda input_shape: Sequential([ #different coding
    #    LSTM(units=64, input_shape=(1, look_back)),
    #    Dense(units=1)]),
    'xgb': XGBRegressor(objective='reg:squarederror', n_estimators=50), 
    'rf': RandomForestRegressor(n_estimators=100),
    #'var': VAR, #different coding
    'dt': DecisionTreeRegressor(max_depth = 50),
    #'prophet': Prophet, #different coding
    #'svr': SVR(kernel='rbf', gamma=0.5, C=10, epsilon = 0.05),
    'etr':ExtraTreesRegressor(n_estimators=50),
    #'ada':AdaBoostRegressor(n_estimators=50)
    'knr':KNeighborsRegressor(n_neighbors=50)
}

# model= models['xgb']



# transform a time series dataset into a supervised learning dataset
#creating a matrix where each row represens a time step and each column represents a feauture.
#n_in (number of time steps to use as input for the model)
#n_out (number of time steps to forecast)


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1] # 1 or #of input columns
	#df = DataFrame(data)
	df = DataFrame(data[:,1:]) #exclude the first column which is the time index.
	timestamp=DataFrame(data[:,1]) #dataframe of the time index
	cols = list() #empty list to hold the lagged and forecasted data.
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i)) # lagged input sequence.
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i)) 
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values



# load the dataset
#data = ship_data.values
#data = [[1, 10, 10], [2, 20, 20], [3, 30, 30], [4, 40, 40], [5, 50, 50], [6, 60, 60], [7, 70, 70], [8, 80, 80], [9, 90, 90], [10, 100, 100]]
#ship_data = np.array(data)
#ship_data = pd.DataFrame(data)
#print(ship_data)
#print(ship_data.values)
#result = series_to_supervised(ship_data,1,2) 
#print(result)




# split a univariate dataset into train/test sets
#The input parameters are data (the dataset to split) and n_test (the number of time steps to use for testing)
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]

#The first numpy array returned, represents the training dataset, 
#which consists of all the observations in the original data array except for the last n_test observations.
#The second numpy array returned, represents the test dataset,
#which consists of the last n_test observations of the original data array.

# fit an random forest model and make a one step prediction
def forecast(model_name, train, testX, feautures, n_out):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1*feautures*n_out], train[:,-1*feautures*n_out:]
	#print('random_forest_shapes',trainX.shape,trainy.shape,trainy[0],train.shape)   
	model= models[model_name]
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	#print(yhat.shape,yhat)
	return yhat[0]

#train, test = train_test_split(data, n_test)

#result=forecast(model_name='rf', train=, testX,feautures=1, n_out=1)
#print(yhat)



# walk-forward validation for univariate data
def walk_forward_validation(model_name, data, feautures, n_out, n_test):
	#print(data.shape,data[0])
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	#print('shape',train.shape,test.shape)   
	# seed history with training dataset
	history = [x for x in train]
	#print(history[0])
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1*feautures*n_out], test[i, -1*feautures*n_out:]
		# fit model on history and make a prediction
		yhat = forecast(model_name, history, testX, feautures, n_out)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		output = "(expected,predicted):"
		for j in range(feautures*n_out):
			output = output  + "(" + '{0:.7f}'.format(test[i][j]) +"," + '{0:.7f}'.format(yhat[j]) + ")"
		print(output)
		#print("yhat",i, ":",yhat)       
	# estimate prediction error
	p=np.array(predictions)
	# Mean Absolute Error
	mae = [mean_absolute_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Mean Squared Error
	mse = [mean_squared_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# MAPE- Mean absolute percentage error
	mape = [mean_absolute_percentage_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]    
	# Median absolute error
	MedAE = [median_absolute_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Max error
	MaxE = [max_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# R² score, the coefficient of determination
	r2 = [r2_score(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Explained variance score
	evs = [explained_variance_score(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)] 
	# Information Criteria for Model Selection
	AIC =n_test* np.log(mse) + 2 * 1
	SBIC = n_test * np.log(mse) + np.log(n_test) * 1
	HQIC = n_test * np.log(mse) + 2 * np.log(np.log(n_test)) * 1
	#error = [mean_absolute_error(test[:, -3], p[:,2]),mean_absolute_error(test[:, -2], p[:,0]),mean_absolute_error(test[:, -1], p[:,1])]
	return [mae,mse,mape,MedAE,MaxE,r2,evs,AIC,SBIC,HQIC], test[:,-1*feautures*n_out:], predictions

def predict(model_name, values, n_in, n_out, n_test):
	# load the dataset
	values = ship_data.values
	#values = float_array.values # VAR --else raws.values
	#values =np.hstack(values, input_column)
	# transform the time series data into supervised learning
	data = series_to_supervised(values, n_in, n_out)    
	feautures =  values.shape[1]-1
	#model = sm.tsa.vector_ar.var_model.VAR(endog=data)
	#model.set_default_exog_names(0) # VAR set the exog_names attribute to zero
	# evaluate
	return walk_forward_validation(model_name, data, feautures, n_out, n_test)


# HOMEMADE DEF

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def print_everything(model_name, values, n_in, n_out, n_test):
    print("\nThe name of the model is:", model_name)
    model= models[model_name]
    print("\nExpected vs Predicted:")
    error , y , yhat  = predict(model_name, values, n_in, n_out, n_test)
    
    print("\nThe error values for", model_name,":")
    MAE = ", ".join(["%.10f" % member for member in error[0]])
    MSE = ", ".join(["%.10f" % member for member in error[1]])
    mape = ", ".join(["%.10f" % member for member in error[2]])
    MedAE = ", ".join(["%.10f" % member for member in error[3]])
    MaxE = ", ".join(["%.10f" % member for member in error[4]])
    r2 = ", ".join(["%.5f" % member for member in error[5]]) # it doesnt really apply in this
    evs = ", ".join(["%.5f" % member for member in error[6]])
    AIC =", ".join(["%.5f" % member for member in error[7]])
    SBIC =", ".join(["%.5f" % member for member in error[8]])
    HQIC =", ".join(["%.5f" % member for member in error[9]])
    #SBIC
    #HQIC
    print(f"\nMAE = {MAE} \nMSE = {MSE} \nmape = {mape} \nMedAE = {MedAE} \nMaxE = {MaxE} \nr2 = {r2} \nevs = {evs}")
    print("\nThe Information Criteria for", model_name,":")
    print(f"\nAIC = {AIC}  \nSBIC = {SBIC} \nHQIC = {HQIC}")
    print("\n\n")
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))
    plt.subplots_adjust(hspace=0.4)

    axs[0].set_title("Expected vs Predicted Long")
    axs[0].set_xlabel("Sample")
    axs[0].set_ylabel("Long")
    axs[0].plot(np.array(y)[:, 0], label="Expected Long")
    axs[0].plot(np.array(yhat)[:, 0], label="Predicted Long")
    ymin = min(np.min(np.array(y)[:, 0]), np.min(np.array(yhat)[:, 0]))
    ymax = max(np.max(np.array(y)[:, 0]), np.max(np.array(yhat)[:, 0]))
    axs[0].set_ylim(ymin, ymax)
    axs[0].legend()

    axs[1].set_title("Expected vs Predicted Lat")
    axs[1].set_xlabel("Sample")
    axs[1].set_ylabel("Lat")
    axs[1].plot(np.array(y)[:, 1], label="Expected Lat")
    axs[1].plot(np.array(yhat)[:, 1], label="Predicted Lat")
    ymin = min(np.min(np.array(y)[:, 1]), np.min(np.array(yhat)[:, 1]))
    ymax = max(np.max(np.array(y)[:, 1]), np.max(np.array(yhat)[:, 1]))
    axs[1].set_ylim(ymin, ymax)
    axs[1].legend()

    plt.show()
"""

# Code

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import HuberRegressor
#from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Ridge






# Define the models
models = {
    #lstm': lambda input_shape: Sequential([ #different coding
    #    LSTM(units=64, input_shape=(1, look_back)),
    #    Dense(units=1)]),
    'xgb': XGBRegressor(objective='reg:squarederror', n_estimators=50), 
    'rf': RandomForestRegressor(n_estimators=100),
    #'var': VAR, #different coding
    'dt': DecisionTreeRegressor(max_depth = 50),
    #'prophet': Prophet, #different coding
    #'svr': SVR(kernel='rbf', gamma=0.5, C=10, epsilon = 0.05),
    'etr':ExtraTreesRegressor(n_estimators=50),
    #'ada':AdaBoostRegressor(n_estimators=50)
    'knr':KNeighborsRegressor(n_neighbors=50),
    'Lasso':Lasso(max_iter=1000),
    'ElasticNet':ElasticNet(max_iter=1000),
    'krr':KernelRidge(alpha=1.0,kernel='polynomial',degree=4),
    'Ridge':Ridge(alpha=0.3),
    'Huber':HuberRegressor(max_iter=100),
    #'Bagging':BaggingRegressor()
    'Ada':AdaBoostRegressor(),
    'svr_poly' : SVR(kernel="poly", C=100, gamma="auto", degree=3, epsilon=0.1, coef0=1)
}

# model= models['xgb']



# transform a time series dataset into a supervised learning dataset
#creating a matrix where each row represens a time step and each column represents a feauture.
#n_in (number of time steps to use as input for the model)
#n_out (number of time steps to forecast)


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1] # 1 or #of input columns
	#df = DataFrame(data)
	df = DataFrame(data[:,1:]) #exclude the first column which is the time index.
	timestamp=DataFrame(data[:,1]) #dataframe of the time index
	cols = list() #empty list to hold the lagged and forecasted data.
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i)) # lagged input sequence.
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i)) 
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values



# load the dataset
#data = ship_data.values
#data = [[1, 10, 10], [2, 20, 20], [3, 30, 30], [4, 40, 40], [5, 50, 50], [6, 60, 60], [7, 70, 70], [8, 80, 80], [9, 90, 90], [10, 100, 100]]
#ship_data = np.array(data)
#ship_data = pd.DataFrame(data)
#print(ship_data)
#print(ship_data.values)
#result = series_to_supervised(ship_data,1,2) 
#print(result)




# split a univariate dataset into train/test sets
#The input parameters are data (the dataset to split) and n_test (the number of time steps to use for testing)
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]

#The first numpy array returned, represents the training dataset, 
#which consists of all the observations in the original data array except for the last n_test observations.
#The second numpy array returned, represents the test dataset,
#which consists of the last n_test observations of the original data array.

# fit an random forest model and make a one step prediction
def forecast(model_name, train, testX, feautures, n_out):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1*feautures], train[:,-1*feautures:]
	#print('random_forest_shapes',trainX.shape,trainy.shape,trainy[0],train.shape)   
	model= models[model_name]
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	#print(yhat.shape,yhat)
	return yhat[0]

#train, test = train_test_split(data, n_test)

#result=forecast(model_name='rf', train=, testX,feautures=1, n_out=1)
#print(yhat)



# walk-forward validation for univariate data
def walk_forward_validation(model_name, data, feautures, n_out, n_test):
	#print(data.shape,data[0])
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	#print('shape',train.shape,test.shape)   
	# seed history with training dataset
	history = [x for x in train]
	#print(history[0])
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1*feautures], test[i, -1*feautures:]
		# fit model on history and make a prediction
		yhat = forecast(model_name, history, testX, feautures, n_out)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		output = "(expected,predicted):"
		for j in range(feautures):
			output = output  + "(" + '{0:.7f}'.format(test[i][j]) +"," + '{0:.7f}'.format(yhat[j]) + ")"
		print(output)
		#print("yhat",i, ":",yhat)       
	# estimate prediction error
	p=np.array(predictions)
	# Mean Absolute Error
	mae = [mean_absolute_error(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)]
	# Mean Squared Error
	mse = [mean_squared_error(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)]
	# MAPE- Mean absolute percentage error
	mape = [mean_absolute_percentage_error(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)]    
	# Median absolute error
	MedAE = [median_absolute_error(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)]
	# Max error
	MaxE = [max_error(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)]
	# R² score, the coefficient of determination
	r2 = [r2_score(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)]
	# Explained variance score
	evs = [explained_variance_score(test[:,-1*feautures+j],p[:,j]) for j in range(feautures)] 
	# Information Criteria for Model Selection
	AIC =n_test* np.log(mse) + 2 * 1
	SBIC = n_test * np.log(mse) + np.log(n_test) * 1
	HQIC = n_test * np.log(mse) + 2 * np.log(np.log(n_test)) * 1
	#error = [mean_absolute_error(test[:, -3], p[:,2]),mean_absolute_error(test[:, -2], p[:,0]),mean_absolute_error(test[:, -1], p[:,1])]
	return [mae,mse,mape,MedAE,MaxE,r2,evs,AIC,SBIC,HQIC], test[:,-1*feautures:], predictions

def predict(model_name, values, n_in, n_out, n_test):
	# load the dataset
	values = ship_data.values
	#values = float_array.values # VAR --else raws.values
	#values =np.hstack(values, input_column)
	# transform the time series data into supervised learning
	data = series_to_supervised(values, n_in, n_out)    
	feautures =  values.shape[1]-1
	#model = sm.tsa.vector_ar.var_model.VAR(endog=data)
	#model.set_default_exog_names(0) # VAR set the exog_names attribute to zero
	# evaluate
	return walk_forward_validation(model_name, data, feautures, n_out, n_test)


# HOMEMADE DEF

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def print_everything(model_name, values, n_in, n_out, n_test):
    print("\nThe name of the model is:", model_name)
    model= models[model_name]
    print("\nExpected vs Predicted:")
    error , y , yhat  = predict(model_name, values, n_in, n_out, n_test)
    
    print("\nThe error values for", model_name,":")
    MAE = ", ".join(["%.10f" % member for member in error[0]])
    MSE = ", ".join(["%.10f" % member for member in error[1]])
    mape = ", ".join(["%.10f" % member for member in error[2]])
    MedAE = ", ".join(["%.10f" % member for member in error[3]])
    MaxE = ", ".join(["%.10f" % member for member in error[4]])
    r2 = ", ".join(["%.5f" % member for member in error[5]]) # it doesnt really apply in this
    evs = ", ".join(["%.5f" % member for member in error[6]])
    AIC =", ".join(["%.5f" % member for member in error[7]])
    SBIC =", ".join(["%.5f" % member for member in error[8]])
    HQIC =", ".join(["%.5f" % member for member in error[9]])
    #SBIC
    #HQIC
    print(f"\nMAE = {MAE} \nMSE = {MSE} \nmape = {mape} \nMedAE = {MedAE} \nMaxE = {MaxE} \nr2 = {r2} \nevs = {evs}")
    print("\nThe Information Criteria for", model_name,":")
    print(f"\nAIC = {AIC}  \nSBIC = {SBIC} \nHQIC = {HQIC}")
    print("\n\n")
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))
    plt.subplots_adjust(hspace=0.4)

    axs[0].set_title("Expected vs Predicted Long")
    axs[0].set_xlabel("Sample")
    axs[0].set_ylabel("Long")
    axs[0].plot(np.array(y)[:, 0], label="Expected Long")
    axs[0].plot(np.array(yhat)[:, 0], label="Predicted Long")
    ymin = min(np.min(np.array(y)[:, 0]), np.min(np.array(yhat)[:, 0]))
    ymax = max(np.max(np.array(y)[:, 0]), np.max(np.array(yhat)[:, 0]))
    axs[0].set_ylim(ymin, ymax)
    axs[0].legend()

    axs[1].set_title("Expected vs Predicted Lat")
    axs[1].set_xlabel("Sample")
    axs[1].set_ylabel("Lat")
    axs[1].plot(np.array(y)[:, 1], label="Expected Lat")
    axs[1].plot(np.array(yhat)[:, 1], label="Predicted Lat")
    ymin = min(np.min(np.array(y)[:, 1]), np.min(np.array(yhat)[:, 1]))
    ymax = max(np.max(np.array(y)[:, 1]), np.max(np.array(yhat)[:, 1]))
    axs[1].set_ylim(ymin, ymax)
    axs[1].legend()

    plt.show()
"""

In [ ]:
"""
print("svr_poly, lag=1, n_out=1, test=1000")
print_everything('svr_poly',ship_data, 1, 1, 10)
#print("xgb - 5 - 1 - 1000")
#print_everything('xgb', ship_data, 5, 1, 1000)
#print("dt - 5 - 1 - 1000")
#print_everything('dt', ship_data, 5, 1, 1000)
"""

## LSTM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

df = pd.read_csv("/kaggle/input/aisbrest/ais_brest_locations.csv")

ship_data = df[df['id'] ==227635210] #22347
#ship_data = df[df['id'] == 245334000] #10.000

ship_data = ship_data[ship_data['latitude'] <= 50]
ship_data = ship_data[ship_data['longitude'] >= -5]
# Select only the desired columns and sort by timestamp
#desired_columns = ['timestamp','longitude', 'latitude']
ship_data = ship_data.sort_values(by='timestamp')
desired_columns = ['longitude']
ship_data = ship_data[desired_columns]

print(ship_data.values)
plt.plot(ship_data.values)


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

df = pd.read_csv("/kaggle/input/aisbrest/ais_brest_locations.csv")

ship_data = df[df['id'] ==227635210] #22347
#ship_data = df[df['id'] == 245334000] #10.000

ship_data = ship_data[ship_data['latitude'] <= 50]
ship_data = ship_data[ship_data['longitude'] >= -5]
# Select only the desired columns and sort by timestamp
#desired_columns = ['timestamp','longitude', 'latitude']
ship_data = ship_data.sort_values(by='timestamp')
desired_columns = ['longitude']
ship_data = ship_data[desired_columns]

#ship_data=timeseries


# train-test split for time series
train_size = int(len(ship_data) * 0.82052)
test_size = len(ship_data) - train_size
train, test = ship_data[:train_size], ship_data[train_size:]


def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback].astype(np.float64)
        target = dataset[i+1:i+lookback+1].astype(np.float64)
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)




lookback = 1
#X_train, y_train = create_dataset(train, lookback=lookback)
#X_test, y_test = create_dataset(test, lookback=lookback)

X_train, y_train = create_dataset(train.values, lookback=lookback)
X_test, y_test = create_dataset(test.values, lookback=lookback)


class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x = x.to(torch.float64)  # Convert the input tensor to float64
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x


model = AirModel()
model = model.double()  # Convert the model's parameters to double precision
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(ship_data) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(ship_data) * np.nan
    test_plot[train_size+lookback:len(ship_data)] = model(X_test)[:, -1, :]
# plot
#plt.plot(ship_data)
plt.plot(ship_data.values)
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()

## Regressors with different shape y

        """
    
    from sklearn.linear_model import BayesianRidge
    from sklearn.linear_model import SGDRegressor
    from lightgbm import LGBMRegressor

    
    'Bayesian':BayesianRidge(n_iter=100),
    'SGDRegressor':SGDRegressor(max_iter=1000),
     'LGBM':LGBMRegressor()

    
    """

In [ ]:
"""
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(10, 10))

# define gridspec
gs = GridSpec(nrows=3, ncols=2, height_ratios=[1, 1, 2])
gs.update(wspace=0.4, hspace=0.4)

# create the list of values
mae = ["%.10f" % member for member in error[0]]
MSE = ["%.10f" % member for member in error[1]]
mape = ["%.10f" % member for member in error[2]]
MedAE = ["%.10f" % member for member in error[3]]
MaxE =  ["%.10f" % member for member in error[4]]
r2 = ["%.5f" % member for member in error[5]]
evs =["%.5f" % member for member in error[6]]

# convert the values to floats
mae = [float(v) for v in mae]
MSE = [float(v) for v in MSE]
mape = [float(v) for v in mape]
MedAE = [float(v) for v in MedAE]
MaxE = [float(v) for v in MaxE]
r2 = [float(v) for v in r2]
evs = [float(v) for v in evs]

# create the x-axis values (index positions)
indices = list(range(len(mae)))
indices = list(range(len(MSE)))
indices = list(range(len(mape)))
indices = list(range(len(MedAE)))
indices = list(range(len(MaxE)))
indices = list(range(len(r2)))
indices = list(range(len(evs)))

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(15, 6))

# create the histogram
axs[0,0].bar(indices, mae)
axs[0,1].bar(indices, MSE)
axs[0,2].bar(indices, mape)
axs[0,3].bar(indices, MedAE)
axs[1,0].bar(indices, MaxE)
axs[1,1].bar(indices, r2)
axs[1,2].bar(indices, evs)

#axs[0].hist(mae, bins=10)
#axs[1].hist(MSE, bins=10)
#axs[2].hist(mape, bins=10)
#axs[3].hist(MedAE, bins=10)
#axs[4].hist(MaxE, bins=10)
#axs[5].hist(r2, bins=10)
#axs[6].hist(evs, bins=10)

#axs[0,0] = fig.add_subplot(gs[0, 0])
axs[0,0].set(xlabel='MAE Values', ylabel='Frequency', title='MAE')
#axs[1] = fig.add_subplot(gs[0, 1])
axs[0,1].set(xlabel='MSE Values', ylabel='Frequency', title='MSE')
#axs[2] = fig.add_subplot(gs[1, 0])
axs[0,2].set(xlabel='Mape Values', ylabel='Frequency', title='Mape')
#axs[3] = fig.add_subplot(gs[1, 1])
axs[0,3].set(xlabel='MedAE Values', ylabel='Frequency', title='MedAE')
#axs[4] = fig.add_subplot(gs[2, :])
axs[1,0].set(xlabel='MaxE Values', ylabel='Frequency', title='MaxE')
axs[1,1].set(xlabel='R2 Values', ylabel='Frequency', title='R2')
axs[1,2].set(xlabel='EVS Values', ylabel='Frequency', title='EVS')

fig.subplots_adjust(left=0.06, bottom=0.1, right=0.95, top=0.95, wspace=0.6, hspace=0.5)

plt.show()
"""

# test 2

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm


# Define the models
models = {
    'lstm': lambda input_shape: Sequential([ #different coding
        LSTM(units=64, input_shape=(1, look_back)),
        Dense(units=1)
    ]),
    'xgb': XGBRegressor(objective='reg:squarederror', n_estimators=25), 
    'rf': RandomForestRegressor(n_estimators=100),
    'var': VAR, #different coding
    'dt': DecisionTreeRegressor(max_depth = 20),
    'prophet': Prophet, #different coding
    'svr': SVR(kernel='rbf', gamma=0.5, C=10, epsilon = 0.05)
}

# model= models['xgb']



# transform a time series dataset into a supervised learning dataset
#creating a matrix where each row represens a time step and each column represents a feauture.
#n_in (number of time steps to use as input for the model)
#n_out (number of time steps to forecast)


def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1] # 1 or #of input columns
	df = DataFrame(data)
	#df = DataFrame(data[:,1:]) #exclude the first column which is the time index.
	#timestamp=DataFrame(data[:,1]) #dataframe of the time index
	cols = list() #empty list to hold the lagged and forecasted data.
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i)) # lagged input sequence.
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i)) 
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values



# load the dataset
#data = ship_data.values
#data = [[1, 10], [2, 20], [3, 30], [4, 40], [5, 50], [6, 60], [7, 70], [8, 80], [9, 90], [10, 100]]
#data = np.array(data)
data = [x for x in range(10)]
print(data)
result = series_to_supervised(data,1,1) 
print(result)
"""

# Time Series Forecasting - Diefferent Horizons

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm


# Define the models
models = {
    'lstm': lambda input_shape: Sequential([ #different coding
        LSTM(units=64, input_shape=(1, look_back)),
        Dense(units=1)
    ]),
    'xgb': XGBRegressor(objective='reg:squarederror', n_estimators=25), 
    'rf': RandomForestRegressor(n_estimators=100),
    'var': VAR #different coding
    'dt': DecisionTreeRegressor(max_depth = 20),
    'prophet': Prophet, #different coding
    'svr': SVR(kernel='rbf', gamma=0.5, C=10, epsilon = 0.05)
}

# model= models['xgb']



# transform a time series dataset into a supervised learning dataset
#creating a matrix where each row represens a time step and each column represents a feauture.
#n_in (number of time steps to use as input for the model)
#n_out (number of time steps to forecast)
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1] # 1 or #of input columns
	df = DataFrame(data[:,1:]) #exclude the first column which is the time index.
	timestamp=DataFrame(data[:,1]) #dataframe of the time index
	cols = list() #empty list to hold the lagged and forecasted data.
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i)) # lagged input sequence.
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values
	print(agg.values)


	# load the dataset
values = ship_data.values
	#values = float_array.values # VAR --else raws.values
	#values =np.hstack(values, input_column)
	# transform the time series data into supervised learning
data = series_to_supervised(values, n_in, n_out)    
	#feautures =  values.shape[1]-1
    
print(n_vars)

# split a univariate dataset into train/test sets
#The input parameters are data (the dataset to split) and n_test (the number of time steps to use for testing)
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]

#The first numpy array returned, represents the training dataset, 
#which consists of all the observations in the original data array except for the last n_test observations.
#The second numpy array returned, represents the test dataset,
#which consists of the last n_test observations of the original data array.

# fit an random forest model and make a one step prediction
def random_forest_forecast(model_name, train, testX, feautures, n_out):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1*feautures*n_out], train[:,-1*feautures*n_out:]
	#print('random_forest_shapes',trainX.shape,trainy.shape,trainy[0],train.shape)
	# fit model
	#model = RandomForestRegressor(n_estimators=1000) #1000 trees -- Change it and see!
	#model=DecisionTreeRegressor(max_depth = 3)    
	model= models[model_name]
	# LSTM Parameters
	#model = Sequential()
	#look_back=1
	#model.add(LSTM(4, input_shape=(1, look_back)))
	#model.add(Dense(1))
	#model.compile(loss='mean_squared_error', optimizer='adam')
	#model.set_default_exog_names(0)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	#print(yhat.shape,yhat)
	return yhat[0]


import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def print_everything(model_name, values, n_in, n_out, n_test):
    print("\nThe name of the model is:", model_name)
    model= models[model_name]
    print("\nExpected vs Predicted:")
    error , y , yhat  = predict(model_name, values, n_in, n_out, n_test)
    
    print("\nThe error values for", model_name)
    MAE = ", ".join(["%.10f" % member for member in error[0]])
    MSE = ", ".join(["%.10f" % member for member in error[1]])
    mape = ", ".join(["%.10f" % member for member in error[2]])
    MedAE = ", ".join(["%.10f" % member for member in error[3]])
    MaxE = ", ".join(["%.10f" % member for member in error[4]])
    r2 = ", ".join(["%.5f" % member for member in error[5]]) # it doesnt really apply in this
    evs = ", ".join(["%.5f" % member for member in error[6]])
    print(f"MAE = {MAE} \nMSE = {MSE} \nmape = {mape} \nMedAE = {MedAE} \nMaxE = {MaxE} \nr2 = {r2} \nevs = {evs}")
    print("\n\n")
    """
 

# Code

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm


# Define the models
models = {
    'lstm': lambda input_shape: Sequential([ #different coding
        LSTM(units=64, input_shape=(1, look_back)),
        Dense(units=1)
    ]),
    'xgb': XGBRegressor(objective='reg:squarederror', n_estimators=25), 
    'rf': RandomForestRegressor(n_estimators=100),
    #'var': VAR #different coding
    'dt': DecisionTreeRegressor(max_depth = 20),
    'prophet': Prophet, #different coding
    'svr': SVR(kernel='rbf', gamma=0.5, C=10, epsilon = 0.05)
}

# model= models['xgb']



# transform a time series dataset into a supervised learning dataset
#creating a matrix where each row represens a time step and each column represents a feauture.
#n_in (number of time steps to use as input for the model)
#n_out (number of time steps to forecast)
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data[:,1:])
	timestamp=DataFrame(data[:,1])
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

# split a univariate dataset into train/test sets
#The input parameters are data (the dataset to split) and n_test (the number of time steps to use for testing)
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]
#The first numpy array returned, represents the training dataset, 
#which consists of all the observations in the original data array except for the last n_test observations.
#The second numpy array returned, represents the test dataset,
#which consists of the last n_test observations of the original data array.

# fit an random forest model and make a one step prediction
def random_forest_forecast(model_name, train, testX, feautures, n_out):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1*feautures*n_out], train[:,-1*feautures*n_out:]
	#print('random_forest_shapes',trainX.shape,trainy.shape,trainy[0],train.shape)
	# fit model
	#model = RandomForestRegressor(n_estimators=1000) #1000 trees -- Change it and see!
	#model=DecisionTreeRegressor(max_depth = 3)    
	model= models[model_name]
	# LSTM Parameters
	#model = Sequential()
	#look_back=1
	#model.add(LSTM(4, input_shape=(1, look_back)))
	#model.add(Dense(1))
	#model.compile(loss='mean_squared_error', optimizer='adam')
	#model.set_default_exog_names(0)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	#print(yhat.shape,yhat)
	return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(model_name, data, feautures, n_out, n_test):
	#print(data.shape,data[0])
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	#print('shape',train.shape,test.shape)   
	# seed history with training dataset
	history = [x for x in train]
	#print(history[0])
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1*feautures*n_out], test[i, -1*feautures*n_out:]
		# fit model on history and make a prediction
		yhat = random_forest_forecast(model_name, history, testX, feautures, n_out)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		output = "(expected,predicted):"
		for j in range(feautures*n_out):
			output = output  + "(" + '{0:.7f}'.format(test[i][j]) +"," + '{0:.7f}'.format(yhat[j]) + ")"
		print(output)
		#print("yhat",i, ":",yhat)
		# summarize progress
		#print('>expected=%.7f, predicted=%.7f, expected=%.6f, predicted=%.6f, expected=%.9f, predicted=%.9f' % (testy[0], yhat[0], testy[1], yhat[1], testy[2], yhat[2]))#change
		#print('inwalk',yhat)
		#print((testy, yhat))        
	# estimate prediction error
	p=np.array(predictions)
	# Mean Absolute Error
	mae = [mean_absolute_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Mean Squared Error
	mse = [mean_squared_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# MAPE- Mean absolute percentage error
	mape = [mean_absolute_percentage_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]    
	# Median absolute error
	MedAE = [median_absolute_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Max error
	MaxE = [max_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# R² score, the coefficient of determination
	r2 = [r2_score(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Explained variance score
	evs = [explained_variance_score(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)] 
	#error = [mean_absolute_error(test[:, -3], p[:,2]),mean_absolute_error(test[:, -2], p[:,0]),mean_absolute_error(test[:, -1], p[:,1])]
	return [mae,mse,mape,MedAE,MaxE,r2,evs], test[:,-1*feautures*n_out:], predictions

def predict(model_name, values, n_in, n_out, n_test):
	# load the dataset
	values = ship_data.values
	#values = float_array.values # VAR --else raws.values
	#values =np.hstack(values, input_column)
	# transform the time series data into supervised learning
	data = series_to_supervised(values, n_in, n_out)    
	feautures =  values.shape[1]-1
	#model = sm.tsa.vector_ar.var_model.VAR(endog=data)
	#model.set_default_exog_names(0) # VAR set the exog_names attribute to zero
	# evaluate
	return walk_forward_validation(model_name, data, feautures, n_out, n_test)


# HOMEMADE DEF

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def print_everything(model_name, values, n_in, n_out, n_test):
    print("\nThe name of the model is:", model_name)
    model= models[model_name]
    print("\nExpected vs Predicted:")
    error , y , yhat  = predict(model_name, values, n_in, n_out, n_test)
    
    print("\nThe error values for", model_name)
    MAE = ", ".join(["%.10f" % member for member in error[0]])
    MSE = ", ".join(["%.10f" % member for member in error[1]])
    mape = ", ".join(["%.10f" % member for member in error[2]])
    MedAE = ", ".join(["%.10f" % member for member in error[3]])
    MaxE = ", ".join(["%.10f" % member for member in error[4]])
    r2 = ", ".join(["%.5f" % member for member in error[5]]) # it doesnt really apply in this
    evs = ", ".join(["%.5f" % member for member in error[6]])
    print(f"MAE = {MAE} \nMSE = {MSE} \nmape = {mape} \nMedAE = {MedAE} \nMaxE = {MaxE} \nr2 = {r2} \nevs = {evs}")
    print("\n\n")
"""

The error message suggests that the input shape of the model is incorrect. Specifically, the expected shape is (None, 1, 1) while the actual shape is (None, 30). This means that the model is expecting a 3D tensor with shape (batch_size, timesteps, features) but it is receiving a 2D tensor with shape (batch_size, features).

One possible solution is to reshape the input data to have the expected shape. You can do this using the reshape() function from NumPy. Here's an example:

In [ ]:
"""
print("rf - 5 - 1 - 1000")
print_everything('rf', ship_data, 5, 1, 1000)
print("xgb - 5 - 1 - 1000")
print_everything('xgb', ship_data, 5, 1, 1000)
print("dt - 5 - 1 - 1000")
print_everything('dt', ship_data, 5, 1, 1000)
"""

In [ ]:
#error , y , yhat  = predict(float_array, 10, 2, 10)  #(raws, 10, 2, 100)
#error , y , yhat  = predict('rf', ship_data, 10, 1, 50)
#print(["%.2f" % member for member in error[0]])

In [ ]:

#MAE = ", ".join(["%.10f" % member for member in error[0]])
#MSE = ", ".join(["%.10f" % member for member in error[1]])
#mape = ", ".join(["%.10f" % member for member in error[2]])
#MedAE = ", ".join(["%.10f" % member for member in error[3]])
#MaxE = ", ".join(["%.10f" % member for member in error[4]])
#r2 = ", ".join(["%.5f" % member for member in error[5]]) # it doesnt really apply in this
#evs = ", ".join(["%.5f" % member for member in error[6]])
#print(f"MAE = {MAE} \nMSE = {MSE} \nmape = {mape} \nMedAE = {MedAE} \nMaxE = {MaxE} \nr2 = {r2} \nevs = {evs}")


In [ ]:
"""
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(10, 10))

# define gridspec
gs = GridSpec(nrows=3, ncols=2, height_ratios=[1, 1, 2])
gs.update(wspace=0.4, hspace=0.4)

# create the list of values
mae = ["%.10f" % member for member in error[0]]
MSE = ["%.10f" % member for member in error[1]]
mape = ["%.10f" % member for member in error[2]]
MedAE = ["%.10f" % member for member in error[3]]
MaxE =  ["%.10f" % member for member in error[4]]
r2 = ["%.5f" % member for member in error[5]]
evs =["%.5f" % member for member in error[6]]

# convert the values to floats
mae = [float(v) for v in mae]
MSE = [float(v) for v in MSE]
mape = [float(v) for v in mape]
MedAE = [float(v) for v in MedAE]
MaxE = [float(v) for v in MaxE]
r2 = [float(v) for v in r2]
evs = [float(v) for v in evs]

# create the x-axis values (index positions)
indices = list(range(len(mae)))
indices = list(range(len(MSE)))
indices = list(range(len(mape)))
indices = list(range(len(MedAE)))
indices = list(range(len(MaxE)))
indices = list(range(len(r2)))
indices = list(range(len(evs)))

fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(15, 6))

# create the histogram
axs[0,0].bar(indices, mae)
axs[0,1].bar(indices, MSE)
axs[0,2].bar(indices, mape)
axs[0,3].bar(indices, MedAE)
axs[1,0].bar(indices, MaxE)
axs[1,1].bar(indices, r2)
axs[1,2].bar(indices, evs)

#axs[0].hist(mae, bins=10)
#axs[1].hist(MSE, bins=10)
#axs[2].hist(mape, bins=10)
#axs[3].hist(MedAE, bins=10)
#axs[4].hist(MaxE, bins=10)
#axs[5].hist(r2, bins=10)
#axs[6].hist(evs, bins=10)

#axs[0,0] = fig.add_subplot(gs[0, 0])
axs[0,0].set(xlabel='MAE Values', ylabel='Frequency', title='MAE')
#axs[1] = fig.add_subplot(gs[0, 1])
axs[0,1].set(xlabel='MSE Values', ylabel='Frequency', title='MSE')
#axs[2] = fig.add_subplot(gs[1, 0])
axs[0,2].set(xlabel='Mape Values', ylabel='Frequency', title='Mape')
#axs[3] = fig.add_subplot(gs[1, 1])
axs[0,3].set(xlabel='MedAE Values', ylabel='Frequency', title='MedAE')
#axs[4] = fig.add_subplot(gs[2, :])
axs[1,0].set(xlabel='MaxE Values', ylabel='Frequency', title='MaxE')
axs[1,1].set(xlabel='R2 Values', ylabel='Frequency', title='R2')
axs[1,2].set(xlabel='EVS Values', ylabel='Frequency', title='EVS')

fig.subplots_adjust(left=0.06, bottom=0.1, right=0.95, top=0.95, wspace=0.6, hspace=0.5)

plt.show()
"""

In [ ]:
"""
fig, axs = plt.subplots(2, 3, figsize=(15, 8))

# subplot 1
p=np.array(yhat)
axs[0, 0].plot(y[:,0], label='Expected Long')
axs[0, 0].plot(p[:,0], label='Predicted Long')
axs[0, 0].legend()

# subplot 2
axs[0, 1].plot(y[:,1], label='Expected Lat')
axs[0, 1].plot(p[:,1], label='Predicted Lat')
axs[0, 1].legend()

# subplot 3
axs[0, 2].plot(y[:,2], label='Expected Speed')
axs[0, 2].plot(p[:,2], label='Predicted Speed')
axs[0, 2].legend()

# subplot 4
axs[1, 0].plot(y[:,3], label='Expected Long')
axs[1, 0].plot(p[:,3], label='Predicted Long')
axs[1, 0].legend()

# subplot 5
axs[1, 1].plot(y[:,4], label='Expected Lat')
axs[1, 1].plot(p[:,4], label='Predicted Lat')
axs[1, 1].legend()

# subplot 6
axs[1, 2].plot(y[:,5], label='Expected Speed')
axs[1, 2].plot(p[:,5], label='Predicted Speed')
axs[1, 2].legend()

# plot
plt.show()
"""

# SVR

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm


# Define the models
models = {
    'lstm': lambda input_shape: Sequential([ #different coding
        LSTM(units=64, input_shape=(1, look_back)),
        Dense(units=1)
    ]),
    'xgb': XGBRegressor(objective='reg:squarederror', n_estimators=25), 
    'rf': RandomForestRegressor(n_estimators=100),
    #'var': VAR #different coding
    'dt': DecisionTreeRegressor(max_depth = 20),
    'prophet': Prophet, #different coding
    'svr': SVR(kernel='rbf', gamma=0.5, C=10, epsilon = 0.05)
}

# model= models['xgb']



# transform a time series dataset into a supervised learning dataset
#creating a matrix where each row represens a time step and each column represents a feauture.
#n_in (number of time steps to use as input for the model)
#n_out (number of time steps to forecast)
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data[:,1:])
	timestamp=DataFrame(data[:,1])
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

# split a univariate dataset into train/test sets
#The input parameters are data (the dataset to split) and n_test (the number of time steps to use for testing)
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]
#The first numpy array returned, represents the training dataset, 
#which consists of all the observations in the original data array except for the last n_test observations.
#The second numpy array returned, represents the test dataset,
#which consists of the last n_test observations of the original data array.

# fit an random forest model and make a one step prediction
def random_forest_forecast(model_name, train, testX, feautures, n_out):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1*feautures*n_out], train[:,-1*feautures*n_out:]
	#print('random_forest_shapes',trainX.shape,trainy.shape,trainy[0],train.shape)
	# fit model
	#model = RandomForestRegressor(n_estimators=1000) #1000 trees -- Change it and see!
	#model=DecisionTreeRegressor(max_depth = 3)    
	model= models[model_name]
	# LSTM Parameters
	#model = Sequential()
	#look_back=1
	#model.add(LSTM(4, input_shape=(1, look_back)))
	#model.add(Dense(1))
	#model.compile(loss='mean_squared_error', optimizer='adam')
	sc = StandardScaler()
	trainX = sc.fit_transform(X_train)
	X_test = sc.transform(X_test)

	scaler = MinMaxScaler()
	trainX = scaler.fit_transform(trainX)
	trainy = scaler.fit_transform(trainy)   
	#model.set_default_exog_names(0)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	#print(yhat.shape,yhat)
	return yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(model_name, data, feautures, n_out, n_test):
	#print(data.shape,data[0])
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	#print('shape',train.shape,test.shape)   
	# seed history with training dataset
	#history = [x for x in train]
	#print(history[0])
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1*feautures*n_out], test[i, -1*feautures*n_out:]
		# fit model on history and make a prediction
		yhat = random_forest_forecast(model_name,train,testX, feautures, n_out) #history,
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		#history.append(test[i])
		output = "(expected,predicted):"
		for j in range(feautures*n_out):
			output = output  + "(" + '{0:.7f}'.format(test[i][j]) +"," + '{0:.7f}'.format(yhat[j]) + ")"
		print(output)
		#print("yhat",i, ":",yhat)
		# summarize progress
		#print('>expected=%.7f, predicted=%.7f, expected=%.6f, predicted=%.6f, expected=%.9f, predicted=%.9f' % (testy[0], yhat[0], testy[1], yhat[1], testy[2], yhat[2]))#change
		#print('inwalk',yhat)
		#print((testy, yhat))        
	# estimate prediction error
	p=np.array(predictions)
	# Mean Absolute Error
	mae = [mean_absolute_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Mean Squared Error
	mse = [mean_squared_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# MAPE- Mean absolute percentage error
	mape = [mean_absolute_percentage_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]    
	# Median absolute error
	MedAE = [median_absolute_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Max error
	MaxE = [max_error(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# R² score, the coefficient of determination
	r2 = [r2_score(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)]
	# Explained variance score
	evs = [explained_variance_score(test[:,-1*feautures*n_out+j],p[:,j]) for j in range(feautures*n_out)] 
	#error = [mean_absolute_error(test[:, -3], p[:,2]),mean_absolute_error(test[:, -2], p[:,0]),mean_absolute_error(test[:, -1], p[:,1])]
	return [mae,mse,mape,MedAE,MaxE,r2,evs], test[:,-1*feautures*n_out:], predictions

def predict(model_name, values, n_in, n_out, n_test):
	# load the dataset
	values = ship_data.values
	#values = float_array.values # VAR --else raws.values
	#values =np.hstack(values, input_column)
	# transform the time series data into supervised learning
	data = series_to_supervised(values, n_in, n_out)    
	feautures =  values.shape[1]-1
	#model = sm.tsa.vector_ar.var_model.VAR(endog=data)
	#model.set_default_exog_names(0) # VAR set the exog_names attribute to zero
	# evaluate
	return walk_forward_validation(model_name, data, feautures, n_out, n_test)


# HOMEMADE DEF

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def print_everything(model_name, values, n_in, n_out, n_test):
    print("\nThe name of the model is:", model_name)
    model= models[model_name]
    print("\nExpected vs Predicted:")
    error , y , yhat  = predict(model_name, values, n_in, n_out, n_test)
    
    print("\nThe error values for", model_name)
    MAE = ", ".join(["%.10f" % member for member in error[0]])
    MSE = ", ".join(["%.10f" % member for member in error[1]])
    mape = ", ".join(["%.10f" % member for member in error[2]])
    MedAE = ", ".join(["%.10f" % member for member in error[3]])
    MaxE = ", ".join(["%.10f" % member for member in error[4]])
    r2 = ", ".join(["%.5f" % member for member in error[5]]) # it doesnt really apply in this
    evs = ", ".join(["%.5f" % member for member in error[6]])
    print(f"MAE = {MAE} \nMSE = {MSE} \nmape = {mape} \nMedAE = {MedAE} \nMaxE = {MaxE} \nr2 = {r2} \nevs = {evs}")
    print("\n\n")
"""

In [ ]:
#print("svr - 10 - 1 - 10")
#print_everything('svr', ship_data, 10, 1, 10)

In [ ]:
"""
import numpy as np
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.vector_ar.var_model import VAR
from prophet import Prophet
from sklearn.svm import SVR
#import statsmodels.api as sm
import datetime


###############################################################################################################
# Load the CSV file into a pandas dataframe
df = pd.read_csv("/kaggle/input/aisbrest/ais_brest_locations.csv")

# Fetch all rows where the ship ID is 227362110
ship_data = df[df['id'] == 227362110]


# Select only the desired columns and sort by timestamp
desired_columns = ['timestamp','longitude', 'latitude','speed']
ship_data = ship_data[desired_columns].sort_values(by='timestamp')

#print(ship_data)
#print(ship_data.timestamp)
#print(ship_data.timestamp.tail(50))

print(ship_data.shape)
test_len = 100
train_len = 1908

train = ship_data.iloc[:train_len, :][['longitude', 'latitude', 'speed']]
test = ship_data.iloc[-test_len:, :][['longitude', 'latitude', 'speed']]


print('Training data shape: ', train.shape)
print('Test data shape: ', test.shape)

#print(train)
#print(test)

###############################################################################################################
scaler = MinMaxScaler()
train = scaler.fit_transform(train)

test = scaler.transform(test)

# Converting to numpy arrays
train_data = train.values
test_data = test.values

timesteps=5

train_data_timesteps=np.array([[j for j in train_data[i:i+timesteps]] for i in range(0,len(train_data)-timesteps+1)])[:,:,0]
train_data_timesteps.shape
"""

# LSTM -Multiple Parallel Input and Multi-Step Output

In [ ]:
"""
# multivariate output stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
#in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
#in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
in_seq1 = array(ship_data.longitude)
in_seq2 = array(ship_data.latitude)
in_seq3 = array(ship_data.speed)
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2,in_seq3, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0)
# demonstrate prediction
#x_input = array([[70,75,145], [80,85,165], [90,95,185]])
#x_input = x_input.reshape((1, n_steps, n_features))
#yhat = model.predict(x_input, verbose=0)
#print(yhat)
"""